# Exploring HathiTrust Metadata without NLP or AI/ML

In [ ]:
'''
author: Samuel J. Huskey
'''
# Import the necessary modules
import pandas as pd
import re
from rapidfuzz import process, fuzz
import spacy
from spacy.training.example import Example
from spacy.training.iob_utils import offsets_to_biluo_tags
import time

In [ ]:
# Read in the CSV file with variant names from the DLL Catalog's authority records
variant_names = pd.read_csv('input/variant-names.csv')
# Load the HathiTrust metadata
hathi_raw = pd.read_csv('input/1908698974-1722799169.txt', sep='\t')
# Open the VIAF author data file
viaf_data = pd.read_csv('output/viaf-authors-output.csv')

## Procedural Python and Pandas Operations

In [ ]:
# Make a new dataframe with the required columns
hathidata = hathi_raw[['author','title','imprint','pub_place','rights_date_used','handle_url']]

# Make a VIAF lookup dictionary
viaf_lookup = pd.Series(viaf_data.Identifier.values, index=viaf_data['H2 Text']).to_dict()

print(f"Starting with {len(hathidata)} rows.")

# Procedural Python and Pandas operations
print("Starting procedural Python and Pandas operations.\n")

# Set the start time
start_time = time.time()

hathidata.loc[:,'author'] = (
    hathidata['author']
    .str.strip()        # Trim leading and trailing whitespace
    .str.rstrip('.')    # Trim terminal period
    .str.rstrip(',')    # Trim terminal comma
)

# Match as many records as possible without fuzzy matching
hathidata.loc[:, 'dll_author_id'] = hathidata.loc[:,'author'].map(viaf_lookup)

# Provide the raw list of names as a string
names_to_be_omitted = """Agathias, d. 582
Alciphron
Anacreon
Apollodorus
Apollonius, Dyscolus, 2nd cent
Apollonius, Dyscolus, active 2nd century
Apollonius, Rhodius
Apolodoro de Atenas
Archimedes
Aristophanes
Arrian
Arriano, Flavio
Artemidoro
Athenaeus, of Naucratis
Bacchylides
Bion, of Phlossa near Smyrna
Cassius Dio Cocceianus
Cleomedes
Constantine VII Porphyrogenitus, Emperor of the East, 905-959
Cyril, Saint, Bishop of Jerusalem, approximately 315-386
Demosthenes
Dio Chrysostomus
Diodorus, Siculus
Diogenes Laertius
Dion Casio
Dionisio de Halicarnaso, ca. 60-5 a.C
Dionysius Cisterciensis
Dionysius, of Halicarnassus
Diógenes Laercio
Dión Casio
Elias, of Nisibis, 975-1046
Epictetus
Euclid
Euclides
Euripides
Eusebio de Cesarea, Obispo de Cesarea, ca. 265-ca. 340
Eusebius, of Caesarea, Bishop of Caesarea, ca. 260-ca. 340.
Eustathius, Macrembolites, 12th cent
Galen
Galeno
Gregory, of Nazianzus, Saint
Gregory, of Nyssa, Saint, ca. 335-ca. 394
Hero of Alexandria
Herodian
Herodotus
Heródoto, 484-425 a. C
Hesiod
Iamblichus, approximately 250-approximately 330
Iamblichus, ca. 250-ca. 330
Irenaeus, Saint, Bishop of Lyon
Isocrates
John Chrysostom, Saint, d. 407
John VI Cantacuzenus, Emperor of the East, 1292-1383
Juliano, Emperador de Roma, 331-363
Justin, Martyr, Saint
Justino, Santo, 100?-165?
Libanius
Lydus, Ioannes Laurentius, 490-
Methodius, of Olympus, Saint, -311
Michael, of Ephesus
Nicander, of Colophon
Nicephorus Callistus, ca. 1256-1335
Nicephorus, Blemmydes, 1197-1272
Orpheus
Philo, of Alexandria
Pindar
Pindarus
Plato
Platón, ca. 427-348 a.C
Plotinus
Plutarch
Polyaenus
Procopius
Píndaro, ca. 518-ca. 438 a. C
Quintus, Smyrnaeus, 4th cent
Sappho
Sextus, Empiricus
Simplicio
Simplicius, of Cilicia
Stobaeus
Strabo
Temistio
Teodoreto, Obispo de Ciro
Teofrasto
Themistius
Theocritus
Theon, of Smyrna
Theophilus, Saint, active 2nd century
Theophrastus
Thucydides
Tryphiodorus
Tucídides, ca. 460-ca. 400 a. C
Xenophon
Xenophon, of Ephesus
Yamblico"""

# Split the string by line breaks to create a list
names_to_omit_list = names_to_be_omitted.splitlines()

# Make a new dataframe without those authors
no_greek = hathidata[~hathidata['author'].isin(names_to_omit_list)]

# Set the end time
end_time = time.time()
print(f"Elapsed time in this stage: {end_time - start_time}.\n\n")

# Use the count() method to count the number of rows with a value other than "NaN"
non_nan_count1 = no_greek['dll_author_id'].count()

# Print the result to the screen
print(f"There are {len(hathidata) - non_nan_count1} rows without a DLL identifier after straight matching.\n")

# Make a new dataframe
unreconciled = no_greek[no_greek['dll_author_id'].isna()]
# Use nunique() to count the unique values in the "author" column
unreconciled_authors = unreconciled['author'].nunique()

print(f"Out of the original {hathidata['author'].nunique()} authors, {unreconciled_authors} remain unreconciled.")


## Fuzzy Matching

In [ ]:
# Make a copy of the original dataframe, filtering out any rows with a value in the dll_author_id field 
fuzzy = no_greek[no_greek['dll_author_id'].isna()]

print("Starting fuzzy matching.")
start_time = time.time()

# Function to perform fuzzy matching
def fuzzy_match_author(author, lookup_dict, threshold=90):
    if pd.isna(author):
        return None
    match = process.extractOne(author, lookup_dict.keys(), scorer=fuzz.token_sort_ratio)
    if match and match[1] >= threshold:
        return lookup_dict[match[0]]
    return None

# Apply the fuzzy_match_author function only to the rows where the mask is True
fuzzy.loc[:, 'dll_author_id'] = fuzzy['author'].apply(fuzzy_match_author, args=(viaf_lookup,))

end_time = time.time()
print(f"Elapsed time in this stage: {end_time - start_time}.\n")

# Use the count() method to count the number of rows with a value other than "NaN"
non_nan_count2 = fuzzy['dll_author_id'].count()

# Print the result to the screen
print(f"There are now {len(hathidata) - (non_nan_count1 + non_nan_count2)} rows out of the original {len(hathidata)} without a DLL identifier after straight and fuzzy matching.")

# Make a new dataframe
unreconciled_with_fuzzy = fuzzy[fuzzy['dll_author_id'].isna()]
# Use nunique() to count the unique values in the "author" column
unreconciled_with_fuzzy_authors = unreconciled_with_fuzzy['author'].nunique()

print(f"Out of the {unreconciled_authors} unreconciled authors after the previous process, {unreconciled_with_fuzzy_authors} remain unmatched.")


## Spacy

In [ ]:
# Make a new copy of the dataframe
spacyframe = fuzzy[fuzzy['dll_author_id'].isna()]

print("Starting to apply NER with Spacy.\n")

start_time = time.time()

# Load the trained model
nlp = spacy.load("custom_ner_model")

# Make a DLL ID lookup dictionary
dll_lookup = variant_names[['Authorized Name','DLL Identifier']]
dll_lookup = variant_names.set_index('Authorized Name')['DLL Identifier'].to_dict()

# Function to identify named entities and match to DLL identifiers
def identify_and_match_author(nlp, text, lookup_dict):
    if pd.isna(text):
        return None
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            matched_name = ent.text
            return lookup_dict.get(matched_name)
    return None

# Apply the function to the 'author' column and create a new column for DLL identifiers
spacyframe.loc[:,'dll_author_id'] = spacyframe['author'].apply(lambda x: identify_and_match_author(nlp, x, dll_lookup))

end_time = time.time()
print(f"Elapsed time in this stage: {end_time - start_time}.\n")

# Use the count() method to count the number of rows with a value other than "NaN"
non_nan_count3 = spacyframe['dll_author_id'].count()

# Print the result to the screen
print(f"There are now {len(hathidata) - (non_nan_count1 + non_nan_count2 + non_nan_count3)} rows out of the original {len(hathidata)} without a DLL identifier after straight, fuzzy, and NER matching.")

# Make a new dataframe
unreconciled_with_spacy = spacyframe[spacyframe['dll_author_id'].isna()]
unreconciled_with_spacy_authors = unreconciled_with_spacy['author'].nunique()

# Use nunique() to count the unique values in the "author" column

print(f"Out of the {unreconciled_with_fuzzy_authors} unreconciled authors after the previous process, {unreconciled_with_spacy_authors} remain unmatched.")

## Post-processing Analysis

In [ ]:
# Group by author and count unique titles
author_title_counts = unreconciled_with_spacy.groupby('author')['title'].nunique()
print("These are the unreconciled authors and the numbers of rows associated with them:")
# Make sure results aren't truncated
pd.set_option('display.max_rows', None)
print(author_title_counts)

# Filter authors with only one title
authors_with_one_title = author_title_counts[author_title_counts == 1]

# Count the number of such authors
count_of_authors_with_one_title = authors_with_one_title.count()
print(f"\n\nOut of {unreconciled_with_spacy_authors} unreconciled authors, {count_of_authors_with_one_title} are 'singletons'.")
print(f"Percentage of unreconciled authors that are singletons: {count_of_authors_with_one_title/unreconciled_with_spacy_authors}.")


## Further Whittling

There are 4,401 unreconciled authors, and 3,298 of those are apparently "singletons". How many of them are actually the same name, just with subtle differences? It's likely that we're dealing with names that don't have an existing record in the DLL Catalog. Otherwise, they would have had a good chance of being matched in the previous processes.

I'm going to see how far I can whittle down the list by doing some more fuzzy matching. Below, I'll make a list of the "unique" names among the 4,401 unreconciled authors. I'll go line by line, checking to see if the current name closely matches any other name in the list. If it does, I'll put it next to the candidate match, with its probability of being a match.

In [ ]:
# I'm going to write the results to a CSV.
import csv

# Print a list of all the unreconciled names
unreconciled_with_spacy_list = [name for name in unreconciled_with_spacy['author'].to_list()]
unreconciled_with_spacy_set = set(unreconciled_with_spacy_list)
filtered_unreconciled_list = [str(name) for name in unreconciled_with_spacy_set]
sorted_unreconciled_list = sorted(filtered_unreconciled_list)

# Function to find matches for a given name
def find_matches(name, names_list, threshold=80):
    matches = process.extract(name, names_list, scorer=fuzz.token_sort_ratio, limit=10)
    return [(match, score) for match, score, _ in matches if score >= threshold and match != name]

# List to store the CSV rows
csv_rows = []

# Set to keep track of processed names
processed_names = set()

# Populate the list with matches
for name in sorted_unreconciled_list:
    if name not in processed_names:
        matches = find_matches(name, sorted_unreconciled_list)
        if matches:
            row = [name]
            for match, score in matches:
                if match not in processed_names:
                    row.extend([match, score])
                    processed_names.add(match)
            csv_rows.append(row)
        processed_names.add(name)

# Define the header
header = ["Name 1", "Candidate Match 1", "Match 1 Probability", 
          "Candidate Match 2", "Match 2 Probability", 
          "Candidate Match 3", "Match 3 Probability",
          "Candidate Match 4", "Match 4 Probability",
          "Candidate Match 5", "Match 5 Probability",
          "Candidate Match 6", "Match 6 Probability",
          "Candidate Match 7", "Match 7 Probability",
          "Candidate Match 8", "Match 8 Probability",
          "Candidate Match 9", "Match 9 Probability"]

# Write the CSV file
with open("matched_names.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for row in csv_rows:
        writer.writerow(row)

In [ ]:
matches = pd.read_csv('matched_names.csv')

In [ ]:
len(matches)

404 is a lot better than 4,401! It's still a lot, and there are probably some false positives, but this is a good start for creating new authority records to be added to the DLL Catalog.

I'll make a final dataframe of the just the reconciled data.

In [ ]:
hathi_id = no_greek[~no_greek['author'].isin(sorted_unreconciled_list)].reset_index()
fuzzy_id = fuzzy[~fuzzy['author'].isin(sorted_unreconciled_list)].reset_index()
spacy_id = spacyframe[~spacyframe['author'].isin(sorted_unreconciled_list)].reset_index()

final_df = pd.concat([hathi_id,fuzzy_id,spacy_id])
final_df = final_df[~final_df['dll_author_id'].isna()]

In [ ]:
len(final_df)

In [ ]:
final_df['title'].nunique()

In [ ]:
final_df['title'].value_counts()

I'm going to use the 'author' column to build up the list of variant names for authors.

In [ ]:
# Make a new dataframe of the 'author' and 'dll_author_id' columns
author_names = final_df[['author','dll_author_id']]

# Open the variant names from VIAF file
viaf_names = pd.read_csv('output/viaf-authors-output.csv')


In [ ]:
# Rename the columns in the VIAF file
viaf_names = viaf_names.rename(columns={'H2 Text':'author','Identifier':'dll_author_id'})

# Concatenate the name frames
names = pd.concat([author_names,viaf_names])

In [ ]:
names

In [ ]:
names_deduplicated = names[names.duplicated(subset=['author'], keep=False) == False]
print(len(names))
print(len(names_deduplicated))

In [ ]:
names_deduplicated.to_csv('output/deep_names.csv',index=False)

In [44]:
# I want to add in all the name forms already in the DLL.
variant_names = pd.read_csv('input/variant-names.csv')
variant_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3228 entries, 0 to 3227
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   DLL Identifier                  3227 non-null   object 
 1   Authorized Name                 3227 non-null   object 
 2   Short Name                      51 non-null     object 
 3   Author Name English             1252 non-null   object 
 4   Author Name Latin               2197 non-null   object 
 5   Author Name Native Language     1045 non-null   object 
 6   BNE URL                         562 non-null    object 
 7   BNF URL                         1361 non-null   object 
 8   DNB URL                         2068 non-null   object 
 9   ICCU URL                        407 non-null    object 
 10  ISNI URL                        1983 non-null   object 
 11  Other Alternative Name Form(s)  38 non-null     object 
 12  Perseus Name                    82

In [46]:
# Use Pandas' melt function to get a listing of variant names matched with a DLL ID.

columns_to_melt = [
    "Authorized Name",
    "Short Name", 
    "Author Name English", 
    "Author Name Latin", 
    "Author Name Native Language", 
    "Other Alternative Name Form(s)", 
    "Perseus Name"
]

melted_df = pd.melt(
    variant_names, 
    id_vars=["DLL Identifier"], 
    value_vars=columns_to_melt,
    var_name="source_column", 
    value_name="author"
)

# Drop rows with NaN in the "author" column
melted_df = melted_df.dropna(subset=["author"])

# Rename the "DLL Identifier" column
melted_df = melted_df.rename(columns={"DLL Identifier": "dll_author_id"})

# Select only the relevant columns
final_melt = melted_df[["author", "dll_author_id"]]

# Save to CSV
final_melt.to_csv("authors_and_identifiers.csv", index=False)

In [47]:
all_names = pd.concat([names_deduplicated,final_melt])
all_names_deduplicated = all_names[all_names.duplicated(subset=['author'], keep=False) == False]

print(len(all_names))
print(len(all_names_deduplicated))

32043
25638


In [48]:
all_names_deduplicated.to_csv('output/all_names_deduplicated.csv',index=False)